In [1]:
#Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain openai lark tiktoken faiss numpy gym==0.25.1 --upgrade

In [2]:
import os, json
os.environ['JAVA_HOME'] = "/home/yh/anaconda3/envs/webshop/bin/"
os.environ['JVM_PATH'] = "/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so"
# Open the file and load the JSON data
with open("./.api_keys.txt", "r") as file:
    api_keys = json.load(file)

os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]
#os.environ["OPENAI_API_KEY"]

In [3]:
import gc
gc.collect()

380

In [4]:
import os

# Change to a directory
os.chdir('./webshop/baseline_models')


from train_rl import parse_args as webenv_args
import imp
from env import WebEnv 
import webshop.baseline_models.test as test_utils


args = test_utils.args
env = test_utils.env

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/tmp/ipykernel_3865/1366313140.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Products loaded.

Keys cleaned.

Attributes loaded.

100%|███████████████████████████████████████████████████████████████| 1181436/1181436 [00:10<00:00, 111783.37it/s]


164 skipped

Loaded 12087 goals.


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


Total huamn goals considered： range(0, 500)
env loaded
30522
30526


In [5]:
args.bart_path = './ckpts/web_search/checkpoint-3600'
args.image = True
args.model_path = "./ckpts/web_click_image/epoch_9/model.pth"
import torch
bart_model = test_utils.BartForConditionalGeneration.from_pretrained(args.bart_path)
print('bart model loaded', args.bart_path)

config = test_utils.BertConfigForWebshop(image=args.image)
model = test_utils.BertModelForWebshop(config)
model.cuda()
model.load_state_dict(torch.load(args.model_path), strict=False)
print('bert il model loaded', args.model_path)


bart model loaded ./ckpts/web_search/checkpoint-3600


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert il model loaded ./ckpts/web_click_image/epoch_9/model.pth


In [6]:
from expert_action_predictor import ExpertActionPredictor
expert_predictor = ExpertActionPredictor(bart_model, model, test_utils.predict, softmax=False)

In [7]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


from langchain.chat_models import ChatOpenAI
from output_parser import (
    AutoGPTOutputParser,
    BaseAutoGPTOutputParser,
)
from prompt import AutoGPTPrompt
from langchain.chains.llm import LLMChain

import numpy as np
import json

DELIMITER="%%%%"     #Pick an unlikely delimiter used somewhere else

def convert_info(info: dict) -> dict:
    global env
    actions = []
    for a in info['valid']:
        if a.startswith('click[item - '):
            actions.append(f'click[{env.name2asin[a[13:-1]]}]')
        else:
            actions.append(a)
    info['valid'] = actions
    
    if info['image_feat'] is not None:
        numpy_array = info['image_feat'].numpy()  # Convert tensor to NumPy array
        info['image_feat'] = numpy_array.tolist()  # Serialize the NumPy array to JSON

    return info

def search(action: str) -> str:
    global env, bart_model, model
    obs, reward, done, info = env.step(f"search[{action}]")
    info = convert_info(info)
    # assert len(valid_actions) > 0
    
    return "=Observation=\n" + obs  + DELIMITER + json.dumps(info)

def click(action: str) -> str:
    global env, bart_model, model
    obs, reward, done, info = env.step(f"click[{action}]")
    obs_prompt = "=Observation=\n" + obs
    info = convert_info(info)
    
    #if valid_actions[0].startswith("search["):
    #    return obs_prompt + "\n:Valid Actions:\n['search[<QUERY>]']"
    #else:
    #    action_prompt = "\nValid Actions:\n" + str(valid_actions)
    if done: 
        return obs_prompt + "{'reward':%s}" % reward    
    return obs_prompt  + DELIMITER + json.dumps(info)

def process_result(full_msg_log):
    import re
    # print(reward_msg)
    output_parser = AutoGPTOutputParser()

    parsed_action = output_parser.parse(full_msg_log[-1].content)
    if parsed_action.name == "finish":
        reward_msg = full_msg_log[-4].content
        match = re.search(r"'reward':([\d\.]+)", reward_msg)
        if match:
            reward = float(match.group(1))
        else:
            reward = 0.0
        return {"purchase": True, "reward": reward * 10}
    else:
        return {"purchase": False, "reward": 0.0}

def init_vectorstore(embedding_size=1536):
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
    return vectorstore

In [8]:
obs, info = env.reset(0)
action = expert_predictor.predict(obs, info)
print(action)
print(info)
obs, reward, done, info = env.step('search[gluten free, 100% vegan plant based protein shake, soy-free, under $50]')
# print(obs)

action = expert_predictor.predict(obs, info)
print(action)


obs, reward, done, info = env.step('click[B07FYPSNH8]')
#print(info)
convert_info(info)
# print(type(info['image_feat']))
info['image_feat'] = torch.tensor(info['image_feat'], dtype=torch.float32)
action = expert_predictor.predict(obs, info)
print(action)



/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


search[gluten free, 100% vegan plant based protein shake]
{'valid': ['search[plant based protein shake that is soy free]', 'search[gluten free vegan plant based protein shake]', 'search[vegan plant based protein shake]', 'search[gluten free plant based protein shake]', 'search[plant based protein shake gluten free]', 'search[vegan plant based protein shake gluten free]', 'search[protein shake gluten free 100% vegan]', 'search[protein shake that is soy free]', 'search[plant based protein shake]', 'search[protein shake gluten free]', 'search[i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 50.00 dollars]'], 'goal': 'Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 50.00 dollars', 'score': 0, 'estimate_score': 0.0, 'prev_ob': None, 'desc': '', 'feat': ''}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


click[item - owyn - 100% vegan plant-based protein shakes | cold brew coffee, 12 fl oz | dairy-free, gluten-free, soy-free, tree nut-free, egg-free, allergy-free, vegetarian]
click[buy now]


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [120]:
SEARCH_TOOL_SPEC_V1 = """
    Requirement:
    1. You can conduct search if your valid action has 'search[<QUERY>]'. 
    2. You need to come up with the best search query based on the instruction in the Observation.
    3. You need to check your past search queries to avoid repetitive searches.
    4. Try to relax constraint on the search query if you cannot find right items.
    Example:
    After you come up with the best query, output in the format as search[<query>].
    For example, for instruction 'i need a long clip-in hair extension which is natural looking,
    and price lower than 30.00 dollars', the query could be like
    'long clip-in hair extension natural looking, lower than 30.00 dollars'
"""

CLICK_TOOL_SPEC_V1 = """
    Requirement:
    1. You should strictly limit the choises of your actions from the Valid Action list, all in 'click[<input>]' format.
    2. In output JSON, use "click" for "command.name", and use the variable <input> for "command.args.tool_input".  
    3. Please make the purchase if you are running out of actions, indicated in 'loop number:'.
    Example: 
    1. The valid actions has'click[b09mm9ncqq]', you use 'click' as "command.name" and 'b09mm9ncqq' as "command.args.tool_input". 
     The actual item description for 'b09mm9ncqq' can be found in the Observation section. 
    2. The valid actions has 'click[back to search]', you use 'click' as "command.name" and 'back to search' as "command.args.tool_input".
    3. The valid actions has 'click[buy now]', you use click as "command.name" and "buy now" as "command.args.tool_input".

"""

SEARCH_TOOL_SPEC_V2 = """
####
    Requirement: Search when there is a search button. 
    Try different searches if you cannot find any matched items in first few pages.
    Example 1:
        =Observation=
            Instruction: 
            i'd love some help finding some organic beard growth oil that can treat hair loss, and price lower than 40.00 dollars
            [button] Search [button_]
        =Response=
            {
                "thoughts": {
                    "text": "There is only one search button, so I can only search",
                    "reasoning": "...",
                    "plan": "...",
                    "criticism": "..."
                },
                "command": {
                    "name": "search",
                    "args": {
                        "tool_input": "organic beard growth oil, under $40"
                        }
                }
            }
####
"""

CLICK_TOOL_SPEC_V2 = """
####
    Requirement: Click the [button]{content}[button_] with the {content}.
                 You shouldn't click when there is only one search button where you should just search.
    Example 1:
        =Observation=
            Instruction:
            i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars
            [button] Back to Search [button_]
            Page 1 (Total results: 50)
            [button] Next > [button_]

            [button] B07FYPSNH8 [button_]
            OWYN - 100% Vegan Plant-Based Protein Shakes | Cold Brew Coffee, 12 Fl Oz | Dairy-Free, Gluten-Free, Soy-Free, Tree Nut-Free, Egg-Free, Allergy-Free, Vegetarian
            $11.07

            [button] B08BPTT8MX [button_]
            Pediasure Grow & Gain with Immune Support, Kids Protein Shake, 27 Vitamins and Minerals, 7g Protein, Helps Kids Catch Up On Growth, Non-GMO, Gluten-Free, Chocolate, 8 Fl Oz, 16 Count
            $100.0
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": "I can only click 'Next >' to got to the next page, or click an item like B07FYPSNH8",
                "plan": "...",
                "criticism": "I shouldn't click "Buy Now" as there's no such [button]."
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "B07FYPSNH8"
                }
            }
        }
    Example 2:
        =Observation=
            Instruction:
            i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars
            [button] Back to Search [button_]
            [button] < Prev [button_]
            OWYN - 100% Vegan Plant-Based Protein Shakes | Cold Brew Coffee, 12 Fl Oz | Dairy-Free, Gluten-Free, Soy-Free, Tree Nut-Free, Egg-Free, Allergy-Free, Vegetarian
            Price: $11.07
            Rating: N.A.
            [button] Description [button_]
            [button] Features [button_]
            [button] Reviews [button_]
            [button] Buy Now [button_]
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": " I can only click 'Back to Search' to search again, or '< Prev' to got to the previous page,
            or 'Description','Features', 'Reviews' to check details, or 'Buy Now' to make the purchase.",
                "plan": "Here the item description matches the instruction and there's no other option to choose, I will click 'Buy Now'.",
                "criticism": "N/A"
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "Buy Now"
                }
            }
        }
    Example 3:
        =Observation=
            Instruction:
            hello, i'm looking for a pair of cargo pants for everyday casual wear? but also hiking-friendly. also, i want an orange pair please, and price lower than 50.00 dollars
            [button] Back to Search [button_]
            [button] < Prev [button_]
            color
            [button] orange [button_]
            [button] yellow [button_]
            size
            [button] small [button_]
            [button] medium [button_]
            Mens Casual Cargo Pants Hi Vis Viz Reflective Overalls High Visibility Safe Work Pants Outdoor Hiking Trousers Big and Tall
            Price: $16.79
            Rating: N.A.
            [button] Description [button_]
            [button] Features [button_]
            [button] Reviews [button_]
            [button] Buy Now [button_]
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": "I can only click 'Back to Search' to search again, or '< Prev' to got to the previous page, or a color or size option to select the specific product option, or 'Description','Features','Reviews' to check details, or 'Buy Now' to make the purchase.",
                "plan": "Here the item description mentions 'organge pair', so I will click 'orange'.",
                "criticism": "N/A."
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "orange"
                }
            }
        }
    Example 4:
        =Observation=
            Instruction:
            I would like a 7 foot oval turquoise rug that is easy to spot clean.
            [button] Back to Search [button_]
            [button] < Prev [button_]
            Transform any living space with the Home Dynamix Premium Area Rug Collection. 
            The versatile and durable Premium area rug collection by Home Dynamix offers style and beauty at an affordable price.
            Instantly elevate your décor to the next level with these designer inspired accent rugs.
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": "I can only click 'Back To Search' or '< Prev'. Since the description matches my instruction,",
                "plan": "I will go back to the item page again by clicking '< Prev'.",
                "criticism": "N/A"
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "< Prev"
                }
            }
        }
####
"""



In [129]:
webshop_search_tool = Tool(
    name = "search",
    func=search,
    description=SEARCH_TOOL_SPEC_V2
)

webshop_click_tool = Tool(
    name = "click",
    func = click,
    description=CLICK_TOOL_SPEC_V2
)

tools = [
    webshop_search_tool,
    webshop_click_tool
]

In [130]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss




In [131]:
ai_name = "Shopping AI"
ai_role = "Web Shopper"
llm = ChatOpenAI(temperature=0.01)
#llm = ChatOpenAI(model_name ="gpt-4", temperature=0.01)


In [132]:
import autogpt
from importlib import reload
reload(autogpt)
from autogpt import AutoGPT


LOOP_LIMIT = 20
input_variables=["memory", "messages", "goals", "user_input"]
expert_predictor = ExpertActionPredictor(bart_model, model, test_utils.predict, softmax=True)
def init_agent(ai_name, ai_role, tools, input_variables,
               llm, vectorstore, init_obs = None, verbose = True, expert_predictor=expert_predictor):
    prompt = AutoGPTPrompt(
                ai_name=ai_name,
                ai_role=ai_role,
                tools=tools,
                input_variables=input_variables,
                token_counter=llm.get_num_tokens
            )
    chain = LLMChain(llm=llm, prompt=prompt)


    agent = AutoGPT(
        ai_name="Shopping Expert",
        memory=vectorstore.as_retriever(),
        chain=chain,
        output_parser=AutoGPTOutputParser(),
        tools=tools,
        loop_limit=LOOP_LIMIT,
        init_obs=init_obs,
        expert_predictor=expert_predictor
    )
    agent.chain.verbose = verbose
    return agent




In [135]:
results = []
full_logs = []
for i in range(22, 24):
    obs, info = env.reset(i)
    print(f"#### Goal {i}: {info['goal']} ####")
    agent = init_agent(ai_name, ai_role, tools, input_variables, llm,
                       init_vectorstore(), init_obs=obs + DELIMITER + json.dumps(info),
                       verbose=False)
    
    goal = f"#### Find the right item and make the right purchase in {LOOP_LIMIT} actions. " + info['goal']
    full_msg_log, actions = agent.run([goal])
    results.append(process_result(full_msg_log))
    full_logs.append(full_msg_log)
    


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


#### Goal 22: Instruction: i want to buy a gray a gray color daybed in twin size with a wooden frame, and price lower than 500.00 dollars ####
loop number:1
Here's one suggestion for the command: search with 'gray a gray color daybed in twin size with a wooden frame'.
Please use your best judgement and feel free to disagree with the example. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will search for 'gray a gray color daybed in twin size with a wooden frame' and look for items with price lower than 500.00 dollars.",
        "reasoning": "I will use the search command to look for the item.",
        "plan": "- Search for 'gray a gray color daybed in twin size with a wooden frame'.\n- Look for items with price lower than 500.00 dollars.",
        "criticism": "N/A"
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "gray a gray color daybed in twin size with a wo

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


loop number:2
Here's one suggestion for the command: click with 'b094yfj95k'.
Please use your best judgement and feel free to disagree with the example. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on the item with ID 'B094YFJ95K' as it matches the description and has a wooden frame.",
        "reasoning": "I will use the click command to select the item.",
        "plan": "- Click on the item with ID 'B094YFJ95K'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B094YFJ95K"
        }
    }
}
Command click returned: =Observation=
Instruction:
i want to buy a gray a gray color daybed in twin size with a wooden frame, and price lower than 500.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
Giantex Wooden Daybed Frame Twin Size, Full Wooden Slats Support, Dual-use Sturdy Sofa Bed for Bedroom Living Roo

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


...

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


loop number:19
Here's one suggestion for the command: click with '< prev'.
Please use your best judgement and feel free to disagree with the example. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will use the click command with the given input.",
        "reasoning": "The item on the page does not match the user's request, so I will go back to the previous page to continue looking.",
        "plan": "- Click on '< Prev'.\n- Continue looking through the search results for an item that matches the user's request.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for contemporary design privacy protected panel for living room, its size should be 52" wide by 90" length, and price lower than 50.00 dollars
[button] Back to Search [button_]
Page 1 (Tota

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


loop number:20
Here's one suggestion for the command: click with 'next >'.
Please use your best judgement and feel free to disagree with the example. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will use the click command with the given input.",
        "reasoning": "The item on the page does not match the user's request, so I will go to the next page to continue looking.",
        "plan": "- Click on 'Next >'.\n- Continue looking through the search results for an item that matches the user's request.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for contemporary design privacy protected panel for living room, its size should be 52" wide by 90" length, and price lower than 50.00 dollars
[button] Back to Search [button_]
Page 2 (Total results

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [134]:
import pandas as pd
import numpy as np
# Convert the dictionary to a DataFrame
df = pd.DataFrame(results, dtype = np.float64)

df_string = df.to_csv(sep='\t', index=False)
print(df_string)

purchase	reward
0.0	0.0
1.0	0.0
0.0	0.0
0.0	0.0
1.0	66.66666666666666
1.0	25.0
1.0	60.0
0.0	0.0
0.0	0.0
0.0	0.0
1.0	3.333333333333333
0.0	0.0
1.0	0.0
1.0	0.0
0.0	0.0
0.0	0.0
1.0	50.0
1.0	3.333333333333333
1.0	33.33333333333333
0.0	0.0



In [128]:
import json

def write_dict_to_file(dictionary, file_path):
    # Open the file in write mode
    with open(file_path, "w") as file:
        for line in dictionary:
            file.write(str(line) + "\n")

# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
file_path = "full_logs_test_item_20_to_39_example_prompt_il.txt"
write_dict_to_file(full_logs, file_path)

In [29]:
#print(full_logs)

[[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars",\n        "reasoning": "I need to find the right item that meets the requirements given by the user",\n        "plan": "- Search for the item using the given requirements\\n- Click on the item if it meets the requirements\\n- If the item is not found, go back to search and try again\\n- If the item is found, click on buy now to make the purchase",\n        "criticism": "I need to be efficient with my searches to ensure I find the right item within the given number of actions"\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "gluten free, 100% vegan plant based protein shake soy-free, price lower than 4

In [15]:
#print(full_logs[-1])

[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for \'pink niuta 2 pack hair towel wrap for dry hair\' and see if I can find it within the price range of 40.00 dollars.",\n        "reasoning": "I will start with a specific search query to narrow down the results and then adjust the query if necessary.",\n        "plan": "- Search for \'pink niuta 2 pack hair towel wrap for dry hair\'\\n- Check if the price is within the range of 40.00 dollars\\n- If not, adjust the search query\\n- Click on the item to view details\\n- If it meets the criteria, click on \'buy now\'",\n        "criticism": "I need to be careful not to waste too many actions on searching and make sure to check the price before clicking on the item."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_inp

In [20]:
for i in range(10, 20):
    obs, info = env.reset(i)
    print(f"############### Goal {i}: {info['goal']} ###############")
    agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore())

    full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])
    results.append(process_result(full_msg_log))
    full_logs.append(full_msg_log)
    print(f"##############################")
    

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


############### Goal 10: Instruction: i am looking fresh scent body lotion for dry skin, and price lower than 30.00 dollars ###############


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking fresh scent body lotion for dry skin, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query 

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking fresh scent body lotion for dry skin, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as search[<query>].
   


> Finished chain.
{
    "thoughts": {
        "text": "I should click on the first item that meets the criteria",
        "reasoning": "The first item that meets the criteria is the best option to choose from. This will save me time and actions.",
        "plan": "- Click on the first item that meets the criteria",
        "criticism": "I need to be careful to ensure that the item meets the criteria before clicking on it."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b087y28xlm"
        }
    }
}


...



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i'm looking for the pendant lights for ceiling lights for living room, and price lower than 90.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as 


> Finished chain.
I am unable to determine the next command as I do not have access to the webpage. Please provide me with the available options.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i'm looking for the pendant lights for ceiling lights for living room, and price lower than 90.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax cons


> Finished chain.
I am unable to determine the next command as I do not have access to the webpage. Please provide me with the available options.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i'm looking for the pendant lights for ceiling lights for living room, and price lower than 90.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax cons


> Finished chain.
{
    "thoughts": {
        "text": "I need to purchase the item as it meets the criteria of being a pendant light for ceiling lights for living room with a price lower than 90.00 dollars.",
        "reasoning": "I have clicked on the item that meets the criteria given by the user and now need to purchase the item.",
        "plan": "- Search for pendant lights for ceiling lights for living room with a price lower than 90.00 dollars.\n- Click on the item that meets the criteria.\n- Purchase the item.",
        "criticism": "I need to make sure that I am purchasing the right item and not wasting any actions."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the


> Finished chain.
Great! Let's use the "finish" command to signal that we have completed our objectives. 

{
    "command": {
        "name": "finish",
        "args": {
            "response": "I have completed my objectives."
        }
    }
}
##############################


In [21]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(results, dtype = np.float64)

df_string = df.to_csv(sep='\t', index=False)
print(df_string)

purchase	reward
1.0	75.0
1.0	66.66666666666666
1.0	66.66666666666666
1.0	50.0
1.0	75.0
1.0	0.0
1.0	50.0
1.0	66.66666666666666
1.0	66.66666666666666
1.0	33.33333333333333
1.0	50.0
1.0	33.33333333333333
1.0	0.0
1.0	0.0
1.0	0.0
1.0	12.5
1.0	25.0
1.0	0.0



In [24]:
import json

def write_dict_to_file(dictionary, file_path):
    # Open the file in write mode
    with open(file_path, "w") as file:
        for line in dictionary:
            file.write(str(line) + "\n")

# Example usage
file_path = "full_logs_test_item_2_to_19.txt"
write_dict_to_file(full_logs, file_path)


# Below is For Testing Purpose

In [24]:
#obs, info = env.reset(0)
#print(info['goal'])

Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [25]:
#agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore())

#full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i


> Finished chain.
{
    "thoughts": {
        "text": "I will click on the first item that meets all the requirements.",
        "reasoning": "The first item that meets all the requirements is the best option to purchase.",
        "plan": "- Click on the first item that meets all the requirements.",
        "criticism": "I need to ensure that the item meets all the requirements before clicking on it."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b07fypsnh8"
        }
    }
}


> Entering new LLMChain chain...


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' i

Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as search[<query>].
  


> Finished chain.
{
    "thoughts": {
        "text": "I will purchase the item since it meets all the requirements and is within the price range.",
        "reasoning": "The item meets all the requirements and is within the price range, so it is the best option to purchase.",
        "plan": "- Purchase the item.",
        "criticism": "I need to ensure that the item is added to the cart and the purchase is successful."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i


> Finished chain.
{
    "thoughts": {
        "text": "I will finish my shopping since I have successfully purchased the item.",
        "reasoning": "Since I have successfully purchased the item, there is no need to continue shopping.",
        "plan": "- Finish my shopping.",
        "criticism": "I need to ensure that the purchase is successful and the item is delivered."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the gluten-free, 100% vegan plant-based protein shake that is soy-free and within the price range. Shopping complete."
        }
    }
}


In [33]:
results = []
result_01 = process_result(full_msg_log)
results.append(result_01)

In [31]:
#obs, info = env.reset(1)
#print(f"############### {info['goal']} ###############")
#agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore(), verbose = False)

#full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


################### Instruction: i need a 120 ml hair mask treatment, and price lower than 140.00 dollars ########################
{
    "thoughts": {
        "text": "I will start by searching for a 120 ml hair mask treatment with a price lower than 140.00 dollars.",
        "reasoning": "I will use the search command to find the hair mask treatment. I will also keep track of my past search queries to avoid repetitive searches.",
        "plan": "- Use the search command to find a 120 ml hair mask treatment with a price lower than 140.00 dollars.\n- Check my past search queries to avoid repetitive searches.\n- If I cannot find the right item, I will relax the constraint on the search query.\n- If I am running out of actions, I will make the purchase.",
        "criticism": "I need to be efficient with my search queries to avoid wasting actions."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "120 ml hair mask treatment price lower than 14

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


{
    "thoughts": {
        "text": "I will click on the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "The Linange Milk and Keratin Mask 500ml hair mask treatment is the only item on the page that meets the requirements of being a 120 ml hair mask treatment with a price lower than 140.00 dollars.",
        "plan": "- Click on the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "criticism": "I need to be careful to ensure that I am clicking on the correct item."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B079HGJ5MH"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


I will click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.

{
    "thoughts": {
        "text": "I will click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "The 'Buy Now' button is the next logical step to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "plan": "- Click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "criticism": "I need to ensure that I am clicking on the correct button to make the purchase."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


The purchase has been made, so I will use the 'finish' command to signal that I have completed all my objectives.

{
    "thoughts": {
        "text": "I have completed the purchase of the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "Since the purchase has been made, there are no further actions required.",
        "plan": "- Use the 'finish' command to signal that I have completed all my objectives.",
        "criticism": "I need to ensure that I have completed all my objectives before using the 'finish' command."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have completed the purchase of the Linange Milk and Keratin Mask 500ml hair mask treatment."
        }
    }
}


In [38]:
print(full_msg_log)

[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for orange cargo pants that are suitable for everyday casual wear and hiking, and cost less than $30.",\n        "reasoning": "I need to find the right item that meets all the requirements specified by the user.",\n        "plan": "- Search for \'orange cargo pants for everyday casual wear and hiking\'\\n- Check the search results for the right item\\n- Click on the item to view more details\\n- If the item meets all the requirements, click on \'buy now\'\\n- If not, click on \'back to search\' and repeat the process",\n        "criticism": "I need to be careful not to waste too many actions on repetitive searches or irrelevant items."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "orange cargo pants for eve

In [1]:
from output_parser import AutoGPTOutputParser
msg = """
I have successfully purchased the hair mask. I will now use the finish command to signal that I have completed all my objectives.

{
    "thoughts": {
        "text": "I have successfully purchased the hair mask. I will now use the finish command to signal that I have completed all my objectives.",
        "reasoning": "I have found the hair mask with the right price and size and have successfully purchased it. I will now use the finish command to signal that I have completed all my objectives.",
        "plan": "- Use the finish command to signal that I have completed all my objectives.",
        "criticism": "None"
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the hair mask and completed all my objectives."
        }
    }
}
"""
output_parser = AutoGPTOutputParser()
output_parser.parse(msg)

AutoGPTAction(name='finish', args={'response': 'I have successfully purchased the hair mask and completed all my objectives.'})

In [28]:
obs, info = env.reset(0)
action = "search[long clip-in hair extension natural looking]"
obs, reward, done, info = env.step(action)
info = convert_info(info)
print(info)



{'valid': ['click[back to search]', 'click[next >]', 'click[b09mm9ncqq]', 'click[b09g2f3hq2]', 'click[b09cgnh5xt]', 'click[b09qmj5kqh]', 'click[b09llrswf4]', 'click[b004rk7i2w]', 'click[b096mm7xm4]', 'click[b07cktxs1j]', 'click[b09s2z2gmj]', 'click[b08zkdqpz3]'], 'goal': 'Instruction: i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars', 'score': 0.0, 'estimate_score': 0.0, 'prev_ob': 'WebShop\nInstruction: \ni need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars\n[button] Search [button_]\n', 'desc': '', 'feat': ''}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [20]:
action = "search[long clip-in hair extension natural looking]"
obs, reward, done, info = env.step(action)
print(obs)
print(info)

Instruction:
i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars
[button] Back to Search [button_]
Page 1 (Total results: 50)
[button] Next > [button_]

[button] B09MM9NCQQ [button_]
Simayixx Christmas Gift Wigs Hair Extensions 24 Inches Synthetic Long Natural Black Thick Clip Straight Hair Stylish Fasionable Fabulous Hair Piece 6 Pieces Popular Hairstyle(E)
$7.83

[button] B09G2F3HQ2 [button_]
Lurrose 6pcs Girls Hair Bow Ribbon Barrettes Hair Bow Ribbon Clip Wig Hair Bow Clips with Alligator Clip and Hair Extensions Hair Accessory for Toddler Girls Children
$11.99

[button] B09CGNH5XT [button_]
Dual Magnetic Lashes, Magnets False Eyelashes, Soft 3D No Glue Fake Lashes Extension with Tweezers, Natural Look Eyelashes Set with 6 Pieces / 3 Pairs /Mini HD Makeup Mirror
$9.99

[button] B09QMJ5KQH [button_]
Messy Bun Scrunchie Hair Bun Extensions Wavy Curly Messy Donut Hairpieces Synthetic Donut Updo Hair Pieces for Women and Girls (black brown)

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [21]:
action = "click[item - simayixx christmas gift wigs hair extensions 24 inches synthetic long natural black thick clip straight hair stylish fasionable fabulous hair piece 6 pieces popular hairstyle(e)]"
obs, reward, done, info = env.step(action)
print(obs)
print(info)
print(type(info))

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)


Instruction:
i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] a [button_]
  [button] b [button_]
  [button] c [button_]
  [button] d [button_]
  [button] e [button_]
  [button] f [button_]
  [button] g [button_]
  [button] h [button_]
  [button] i [button_]
  [button] j [button_]
Simayixx Christmas Gift Wigs Hair Extensions 24 Inches Synthetic Long Natural Black Thick Clip Straight Hair Stylish Fasionable Fabulous Hair Piece 6 Pieces Popular Hairstyle(E)
Price: $7.83
Rating: N.A.
[button] Description [button_]
[button] Features [button_]
[button] Reviews [button_]
[button] Buy Now [button_]

{'valid': ['click[back to search]', 'click[< prev]', 'click[description]', 'click[features]', 'click[reviews]', 'click[buy now]', 'click[a]', 'click[b]', 'click[c]', 'click[d]', 'click[e]', 'click[f]', 'click[g]', 'click[h]', 'click[i]', 'click[j]'], 'goal': 'Instruction: i 

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find(